# Qlib Workflows

In [1]:
import qlib

qlib.init(provider_uri = '../data/qlib-day')

[56186:MainThread](2022-08-24 22:07:41,076) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[56186:MainThread](2022-08-24 22:07:41,331) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[56186:MainThread](2022-08-24 22:07:41,333) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/oak/Repo/Data/qlib-day')}


## Qlib Factor Overview

In this part we try to define some factor using expression engine in qlib, and test the factor performance in the traditional way.

In [36]:
from qlib.contrib.report.analysis_position.score_ic import score_ic_graph
from qlib.contrib.eva.alpha import calc_ic
from qlib.utils import init_instance_by_config
from qlib.data.dataset.processor import CSZScoreNorm, DropnaProcessor

momentum_exp = '$close / Ref($close, 20) - 1'
momentum_name = 'Mom20'
label_exp = 'Ref($close, -11) / Ref($close, -1) - 1'
label_name = 'Forward'

dh = init_instance_by_config({
    "class": "qlib.data.dataset.handler.DataHandlerLP",
    "kwargs": {
        "data_loader": {
            "class": "qlib.data.dataset.loader.QlibDataLoader",
            "kwargs":{
                "config": {
                    "feature": [(momentum_exp,), (momentum_name,)], 
                    "label": [(label_exp,), (label_name,)]
                },
            }
        },
        "instruments": "000300.XSHG",
        "start_time": "2015-01-01",
        "end_time": "2022-08-01",
        "process_type": "append",
        "infer_processors": [CSZScoreNorm(), DropnaProcessor()],
    },
})
data = dh.fetch(data_key="infer")
ic, ric = calc_ic(data.Mom20, data.Forward, dropna=True)

[56186:MainThread](2022-08-24 22:51:59,464) INFO - qlib.timer - [log.py:117] - Time cost: 12.158s | Loading data Done
[56186:MainThread](2022-08-24 22:52:03,043) INFO - qlib.timer - [log.py:117] - Time cost: 3.575s | CSZScoreNorm Done
[56186:MainThread](2022-08-24 22:52:03,055) INFO - qlib.timer - [log.py:117] - Time cost: 0.010s | DropnaProcessor Done
[56186:MainThread](2022-08-24 22:52:03,056) INFO - qlib.timer - [log.py:117] - Time cost: 3.590s | fit & process data Done
[56186:MainThread](2022-08-24 22:52:03,056) INFO - qlib.timer - [log.py:117] - Time cost: 15.750s | Init data Done


In [44]:
import plotly.graph_objects as go

layout = [
    go.Scatter(x=ic.index, y=ic.values, name='Normal IC', mode='lines+markers'), 
    go.Scatter(x=ric.index, y=ric.values, name='Rank IC', mode='lines+markers')
]
go.Figure(data = layout)